In [1]:
import glob
import math
import os

import numpy as np
import pandas as pd

In [2]:
# Set Pandas display option to display all the columns
pd.set_option("display.max_columns", None)

In [3]:
# Generate the dataset and make minor adjustments

df = pd.read_csv("data/biblical_bhsa_dss_without_min.csv").fillna("")
df["dir_he"] = df["dir_he"].astype(str)
df["dir_he"] = df["dir_he"].replace("1.0", "1").replace("0.0", "0")

df["dir_he_dss"] = df["dir_he_dss"].astype(str)
df["dir_he_dss"] = df["dir_he_dss"].replace("1.0", "1").replace("0.0", "0")
df.sort_values(["book", "chapter", "verse_num", "scroll"], ascending=[True, True, True, False], ignore_index=True, inplace=True)

In [4]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'cmpl_translation', 'dir_he', 'cmpl_constr', 'cmpl_nt',
       'cmpl_anim', 'cmpl_det', 'cmpl_indiv', 'cmpl_complex', 'dir_he_dss',
       'sign_info', 'stem', 'tense', 'motion_type', 'comments',
       'preposition_1', 'preposition_2', 'preposition_3', 'preposition_4',
       'preposition_5', 'preposition_6', 'preposition_7'],
      dtype='object')

In [8]:
# Show the set of books present in the dataset
#set(df.book)
#len(set(df.book))

In [10]:
# Create one dataset per book (containing both BHSA and DSS rows)

unique_books = df['book'].unique()

# New columns to add

new_columns = [
    "cmpl_translation",
    "motion_type",
    "comments",
]

# Creating one dataset per unique book in the original dataset
for book in unique_books:
    book_df = df[df['book'] == book].copy()

    # Add each new column with NaN values
    for column in new_columns:
        book_df[column] = ''
        
    # Save the new dataset as a csv file
    book_df.to_csv(f"data/biblical_datasets_without_min/{book}_dataset.csv", index=False)

In [11]:
len(unique_books)

57